## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(["EIN", "NAME"], axis=1, inplace=True)

In [4]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()

print(unique_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()

print(application_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace=["T9","T13","T12","T2","T25","T14","T29","T15","T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()

print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [11]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()>1

print(application_df['CLASSIFICATION'].value_counts()[classification_counts])

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [14]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace=application_df['CLASSIFICATION'].value_counts().index[5:]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [18]:
# Convert categorical data to numeric with `pd.get_dummies`
numerical_df=pd.get_dummies(application_df)
print(numerical_df)

       STATUS   ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0           1      5000              1                       0   
1           1    108590              1                       0   
2           1      5000              0                       0   
3           1      6692              1                       0   
4           1    142590              1                       0   
...       ...       ...            ...                     ...   
34294       1      5000              0                       0   
34295       1      5000              0                       0   
34296       1      5000              0                       0   
34297       1      5000              1                       0   
34298       1  36500179              0                       0   

       APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  \
0                         1                     0                    0   
1                         0                     0          

In [27]:
# Split our preprocessed data into our features and target arrays
X=numerical_df[list(filter(lambda x: x!="IS_SUCCESSFUL",numerical_df.columns))]
Y=numerical_df["IS_SUCCESSFUL"]
# Split the preprocessed data into a training and testing dataset
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)

In [28]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [48]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(80,activation="relu",input_shape=(43,)))

# Second hidden layer
nn.add(tf.keras.layers.Dense(50,activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(30,activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(1,activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 80)                3520      
_________________________________________________________________
dense_13 (Dense)             (None, 50)                4050      
_________________________________________________________________
dense_14 (Dense)             (None, 30)                1530      
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 31        
Total params: 9,131
Trainable params: 9,131
Non-trainable params: 0
_________________________________________________________________


In [51]:
# Compile the model
nn.compile(optimizer="Adam",loss="bce",metrics=[tf.keras.metrics.BinaryAccuracy()])

In [52]:
# Train the model
nn.fit(X_train_scaled,Y_train,batch_size=32,epochs=100)

Epoch 1/100
858/858 [==============================] - 1s 693us/step - loss: 0.5570 - binary_accuracy: 0.7301
Epoch 2/100
858/858 [==============================] - 1s 1ms/step - loss: 0.5512 - binary_accuracy: 0.7302
Epoch 3/100
858/858 [==============================] - 1s 868us/step - loss: 0.5508 - binary_accuracy: 0.7321
Epoch 4/100
858/858 [==============================] - 1s 834us/step - loss: 0.5489 - binary_accuracy: 0.7302
Epoch 5/100
858/858 [==============================] - 1s 1ms/step - loss: 0.5475 - binary_accuracy: 0.7334
Epoch 6/100
858/858 [==============================] - 1s 1ms/step - loss: 0.5469 - binary_accuracy: 0.7327
Epoch 7/100
858/858 [==============================] - 1s 1ms/step - loss: 0.5460 - binary_accuracy: 0.7345
Epoch 8/100
858/858 [==============================] - 1s 1ms/step - loss: 0.5457 - binary_accuracy: 0.7332
Epoch 9/100
858/858 [==============================] - 1s 2ms/step - loss: 0.5447 - binary_accuracy: 0.7337
Epoch 10/100
858/858 [

KeyboardInterrupt: 

In [54]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,Y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5582 - binary_accuracy: 0.7315
Loss: 0.5581756234169006, Accuracy: 0.7314868569374084


In [56]:
# Export our model to HDF5 file
nn.save(r"C:\Users\dasha\OneDrive\Desktop\Challenges\Challenge 21\Starter_Code\AlphabetSoupCharity_Optimization.h5")